In [94]:
import numpy as np
import pandas as pd
import flopy

In [95]:
swt = flopy.seawat.Seawat2("henry_heat",model_ws="temp")

In [96]:
nlay,nrow,ncol = 100,1,200
delz,delc,delr = 10,1,10
top = 0.0
botm = [top - ((i+1) * delz) for i in range(nlay)]
dis = flopy.modflow.ModflowDis(swt,nlay=nlay,nrow=nrow,ncol=ncol,
                              top=top,botm=botm,delr=delr,delc=delc,
                              perlen=[200000],steady=False,nstp=200)

In [97]:
bas = flopy.modflow.ModflowBas(swt,strt=top)
lpf = flopy.modflow.ModflowLpf(swt,hk=10,vka=0.1,ss=1.0e-5)
pcg = flopy.modflow.ModflowPcg(swt)
oc = flopy.modflow.ModflowOc(swt)

In [98]:
itype = flopy.mt3d.Mt3dSsm.itype_dict() 
print(itype)
wel_q = 10.0/nlay
wel_conc = 0.0
wel_temp = 25.0
wel_itype = itype["WEL"]
wel_flow_sp = {0:[(k,0,0,wel_q) for k in range(nlay) ]}
wel_conc_temp_sp = {0:[(k,0,0,wel_conc,wel_itype,wel_conc,wel_temp) for k in range(nlay) ]}
wel = flopy.modflow.ModflowWel(swt,stress_period_data=wel_flow_sp)

{'BAS6': 1, 'MAS': 15, 'CHD': 1, 'GHB': 5, 'RIV': 4, 'WEL': 2, 'CC': -1, 'PBC': 1, 'DRN': 3}


In [99]:
ocean_h = 0.0
ocean_conc = 35.0
ocean_temp = 5.0
chd_flow_sp = {0:[(k,0,ncol-1,ocean_h,ocean_h) for k in range(nlay)]}
chd_conc_temp_sp = {0:[(k,0,ncol-1,ocean_conc,itype["CHD"],
                       ocean_conc,-1) for k in range(nlay)]}
itype_minus1 = [(k,0,ncol-1,ocean_conc,-1,-1,ocean_temp) for k in range(nlay)]
chd_conc_temp_sp[0].extend(itype_minus1)
chd = flopy.modflow.ModflowChd(swt,stress_period_data=chd_flow_sp)

In [100]:
btn = flopy.mt3d.Mt3dBtn(swt,prsity=0.35,ncomp=2,sconc=35.0,sconc2=5.0,mcomp=2)
adv = flopy.mt3d.Mt3dAdv(swt,mixelm=-1,percel=1.0,)
gcg = flopy.mt3d.Mt3dGcg(swt,mxiter=1,iter1=200,isolve=2)
dsp = flopy.mt3d.Mt3dDsp(swt,al=1.0,trpt=0.1,trpv=0.1,multiDiff=True,dmcoef=1.0e-10,dmcoef2=0.15031)

In [101]:
ssm_data = wel_conc_temp_sp
ssm_data[0].extend(chd_conc_temp_sp[0])
ssm = flopy.mt3d.Mt3dSsm(swt,stress_period_data=ssm_data)

In [102]:
rct = flopy.mt3d.Mt3dRct(swt,isothm=1,igetsc=0,rhob=1761.5,sp1=0.0,
                         sp12=2.0e-4)

RCT: setting srconc for component 2 to zero, kwarg name srconc2
RCT: setting sp2 for component 2 to zero, kwarg name sp22
RCT: setting rc1 for component 2 to zero, kwarg name rc12
RCT: setting rc2 for component 2 to zero, kwarg name rc22


In [103]:
vdf = flopy.seawat.SeawatVdf(swt,mtdnconc=-1,mfnadvfd=1,nswtcpl=0,
                             iwtable=0,densemin=0.0,densemax=0.0,
                            densref=1000.0,drhodprhd=0.0,prhrhd=0.0,
                             nsrhoeos=2,drhodc=[0.7,-.3750],
                            crhoref=[0.0,25.0])

In [104]:
vsc = flopy.seawat.SeawatVsc(swt,mt3dmuflg=-1,viscmin=0,viscmax=0,nsmueos=1,viscref=8.904e-4,
                            mutempopt=1,mtmuspec=1,dmudc=1.923e-6,cmuref=0.0,
                            mtmutempspec=2,amucoeff=[239.4e-7,10.0,248.37,133.15])

In [105]:
swt.write_input()

In [106]:
swt.run_model()

FloPy is using the following executable to run the model: /Users/jwhite/bin/swt_v4

                                  SEAWAT Version 4
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 4.00.05 10/19/2012                      

Incorporated MODFLOW Version: 1.18.01 06/20/2008                      
Incorporated MT3DMS  Version: 5.20    10/30/2006                      


 This program is public domain and is released on the
 condition that neither the U.S. Geological Survey nor
 the United States Government may be held liable for any
 damages resulting from their authorized or unauthorized
 use.


 Using NAME file: henry_heat.nam                                                                                                                                                                                          
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2016/06/21 17:32:06


 STRESS PERIOD NO.    1

  STRESS PERIOD    1

(False, [])